## 1.2 Language Models
* Adapt your implementation of logistic regression from Assignment 2 to the implementation of a feedforward neural network (or create a new implementation from scratch). If you use an online implementation as a reference state that in your report and the modifications you have made.
* Your neural network should predict upcoming words from prior word context (see Section 7.5 of the reference book).

In [15]:
import numpy as np
from math import exp
from random import random
from tqdm import tqdm

In [16]:
def forward_ReLU(x):
    return np.maximum(0, x)

def backward_ReLU(x):
    # found on stack overflow
    return (x > 0)

def forward_softmax(x):
    max = np.max(x, axis=0, keepdims=True)
    e_x = np.exp(x - max)
    sum = np.sum(e_x, axis=0, keepdims=True)
    f_x = e_x / sum
    return f_x

In [17]:
class Embedding_Layer:
    """Fully Connected Layer without bias.
       input x weights -> output"""

    def __init__(self, n_input, n_output, seed=1):
        ran = np.random
        ran.seed(seed)
        self.weights = ran.random_sample((n_output, n_input))
        
    def feedforward(self, x):
        return np.matmul(self.weights, x.T)


In [18]:
class Fully_Connected_Layer_Bias:
    """Fully Connected Layer with bias.
       input x weights + bias -> output"""
    
    def __init__(self, n_input, n_output, seed=1):
        ran = np.random
        ran.seed(seed)
        self.weights= ran.random_sample((n_output, n_input))
        self.bias = ran.random_sample(n_output)
    
    def feedforward(self, x):
        x_w = np.matmul(self.weights, x.T)
        x_w_b = x_w + self.bias
        return x_w_b


In [19]:
# 2 layer Neural Network
class Neural_Network:

    # initialize all necessary informatio
    def __init__(self, n_vocab, n_embeddings, learning_rate=0.1, n_prev_words=3):
        
        # information about the model
        self.embedding_layer = Embedding_Layer(n_vocab, n_embeddings)
        self.hidden_layer = Fully_Connected_Layer_Bias(n_embeddings * n_prev_words, n_embeddings)
        self.output_layer = Fully_Connected_Layer_Bias(n_embeddings, n_vocab)
        self.alpha = learning_rate
        
        # z-values representing the results of each layer
        # before activating them, generally means when
        # the input has benn mutiplied by its weights
        self.embedding_z = None
        self.hidden_z =  None
        self.output_z = None
        
        self.embedding_activation = None
        self.hidden_activation = None
        self.output_activation = None
        
        # information about the data
        # is updated when 
        self.vocab = None
        self.one_hot_vectors = None
        self.n_vocab = n_vocab
        self.n_embeddings = n_embeddings
        self.n_prev_words = n_prev_words

    # Maybe use for prediction and encoding data
    def _create_vocab(self, x):
        new_vocab = []
        for word in x:
            if word not in new_vocab:
                new_vocab.append(x)
        self.vocab = new_vocab
        self.one_hot_vectors = [np.zeros(len(self.vocab)) for _ in range(len(self.vocab))]
        for i in range(len(self.vocab)):
            self.one_hot_vectors[i][i] = 1  

    # updates layers forward
    def _feedforward(self, x):
        
        self.x = x
        """Method from 7.5.1 in Speech and Language Processing.
           Step by step commented for easy comparison."""
        e = np.array([self.embedding_layer.feedforward(x[i]) for i in range(self.n_prev_words)])
        # create embedding layer e
        self.embedding_z = np.concatenate(e, axis=0)
        self.embedding_activation = self.embedding_z
        # Multiply by W and pass through ReLU activation function
        self.hidden_z = self.hidden_layer.feedforward(self.embedding_activation)
        self.hidden_activation = forward_ReLU(self.hidden_z)
        
        # Multiply by U and apply softmax reshaping it into |Vocabulary| x 1
        self.output_z = self.output_layer.feedforward(self.hidden_activation)
        self.output_activation = forward_softmax(self.output_z).reshape(1, self.n_vocab)

    def _backprop(self, y):
        # step by step what to dodo
        
        # Find the bias of the output layer
        # derivative of cross entropy: y_pred - y
        self.dL_db_output = (self.output_activation - y) * self.output_activation
        self.dL_dw_output = np.matmul(self.dL_db_output.T, self.hidden_activation.reshape(1, -1))

        output_sum = np.sum(np.matmul(self.dL_db_output, self.output_layer.weights), axis=1, keepdims=True)
        self.hidden_bReLU = backward_ReLU(self.hidden_z)
        self.dL_db_hidden = np.matmul(output_sum.reshape(-1, 1), self.hidden_bReLU.reshape(1, -1))
        self.dL_dw_hidden = np.matmul(self.dL_db_hidden.T, self.embedding_activation.reshape(1, -1))
        
        hidden_sum = np.sum(np.matmul(self.dL_db_hidden, self.hidden_layer.weights), axis=1, keepdims=True)
        delta = np.matmul(hidden_sum.reshape(-1, 1), self.embedding_z.reshape(1, -1))
        q = int(delta.shape[1] / self.n_prev_words)
        self.dL_db_embedding = np.array([delta[:, i: i+q] for i in range(self.n_prev_words)]).reshape(self.n_prev_words, self.n_embeddings)
        self.dL_dw_embedding = np.array([np.matmul(self.dL_db_embedding[i].reshape(-1, 1), self.x[i].reshape(1, -1)) for i in range(self.n_prev_words)])
        
        # finally, update the weights with the new loss gradients
        self._update_weights()
    
    def _update_weights(self):
        # update all the embedding weights
        for i in range(self.n_prev_words):
            self.embedding_layer.weights -= self.alpha * self.dL_dw_embedding[i]
        # update hidden layer
        self.hidden_layer.weights -= self.alpha * self.dL_dw_hidden
        self.hidden_layer.bias -= self.alpha * self.dL_db_hidden.flatten()
        # update output layer
        self.output_layer.weights -= self.alpha * self.dL_dw_output
        self.output_layer.bias -= self.alpha * self.dL_db_output.flatten()

    def fit(self, x, y, n_epochs=1):
        self._feedforward(x)
        self._backprop(y)
            
        return self.output_activation
        
    def predict(self, x):
        # Use logistic regression
        print("TODO")


In [23]:
with open("metamorphosis.txt",'r',encoding='utf-8') as file:
    raw_metamorphosis = file.read()
    
replacements = ['\n', '\t', '\r', ';', ':', '.', ',', '  ']

stripped_metamorphosis = raw_metamorphosis

for rep in replacements:
    stripped_metamorphosis = stripped_metamorphosis.replace(rep, ' ')

stripped_metamorphosis.lower().split()

metamorphosis_vocab = list(set(stripped_metamorphosis))
vocab = {}
one_hots = {}
for i, word in enumerate(metamorphosis_vocab):
    one_hot_vector = np.zeros(len(metamorphosis_vocab))
    one_hot_vector[i] = 1
    vocab[word] = one_hot_vector

print(len(stripped_metamorphosis))

116199


In [26]:
v_len = len(metamorphosis_vocab)
memory_depth = 3
data_set_length = len(stripped_metamorphosis) - memory_depth
batch_size = 20000
model = Neural_Network(v_len, 32)
print(data_set_length, data_set_length // batch_size)
for batch in range(data_set_length // batch_size):
    dataX = np.zeros((batch_size - memory_depth, memory_depth, v_len), dtype='float16')
    dataY = np.zeros((batch_size - memory_depth, v_len), dtype='int16')
    for i in range(batch_size - memory_depth):
        x = np.zeros((memory_depth, len(vocab)))
        for j in range(memory_depth):
            x[j] = vocab[stripped_metamorphosis[(batch*batch_size):(batch*batch_size)+batch_size][i+j]]
        dataX[i] = x
        dataY[i] = vocab[stripped_metamorphosis[(batch*batch_size):(batch*batch_size)+batch_size][i+memory_depth]]
        
    batch_progress = tqdm(total=dataX.shape[0], desc="batch progression {}/{}".format(batch, data_set_length // batch_size))
    for x, y in zip(dataX, dataY):
        result = model.fit(x, y)
        batch_progress.update()
    print(result)
    batch_progress.close()


116196 5


batch progression 0/5: 100%|██████████████████████| 19997/19997 [00:01<00:00, 16469.57it/s]


[[0.0088906  0.00565879 0.00783642 0.04437519 0.00495588 0.00572283
  0.00519349 0.00887587 0.00685393 0.00931377 0.00819821 0.02095734
  0.05796187 0.00682873 0.00857505 0.00747356 0.08132443 0.00720962
  0.0073385  0.00679601 0.00563062 0.00538826 0.00499193 0.00504721
  0.01069428 0.04890444 0.00640441 0.01099156 0.00666486 0.00748014
  0.00959972 0.01100025 0.03934529 0.00852735 0.00792455 0.00683137
  0.06142596 0.0046009  0.00926357 0.00750765 0.00820065 0.0393008
  0.00785332 0.00552585 0.00575352 0.00826956 0.00840439 0.006749
  0.0404274  0.19581078 0.00881982 0.00888992 0.01809751 0.00459549
  0.00960965 0.00580648 0.00669488 0.00705164 0.00557489]]


batch progression 1/5: 100%|██████████████████████| 19997/19997 [00:01<00:00, 16546.73it/s]


[[0.00592361 0.00394747 0.00527226 0.04633366 0.00350556 0.00399601
  0.00373478 0.0065386  0.00592472 0.0061738  0.0068567  0.02682902
  0.07781399 0.00470383 0.00573399 0.00508431 0.09216104 0.00496321
  0.0049852  0.00470273 0.0039314  0.00381627 0.00351615 0.00355114
  0.00970802 0.06261868 0.00445148 0.0195325  0.00457625 0.00508297
  0.0080796  0.00826663 0.04399504 0.00566502 0.00533587 0.00467971
  0.05592976 0.00332479 0.00614183 0.00657821 0.00553133 0.05327202
  0.00529525 0.00386391 0.00401921 0.00640538 0.00559298 0.00462746
  0.05861889 0.1922314  0.00635399 0.00587352 0.02467019 0.00325938
  0.00822338 0.00404334 0.00459912 0.00560592 0.0039475 ]]


...

[[0.00463694 0.00316241 0.00414699 0.04818591 0.00282689 0.00319941
  0.00303801 0.00528024 0.00591282 0.00480107 0.00668329 0.03284603
  0.07577496 0.00373894 0.00451756 0.00401926 0.09667494 0.00394843
  0.00394008 0.00374418 0.00315128 0.00307653 0.00283083 0.00286043
  0.0096455  0.05750098 0.00354457 0.04120202 0.00364216 0.0040252
  0.00751832 0.00718367 0.04839209 0.00443206 0.00421362 0.00371479
  0.06665147 0.00272761 0.0047788  0.00613914 0.00436461 0.04662774
  0.00416359 0.00309856 0.00321707 0.00551663 0.00438278 0.00367455
  0.05776255 0.1947155  0.0053409  0.00458602 0.02870231 0.00263483
  0.00778846 0.00323761 0.00367269 0.00502215 0.00318203]]


batch progression 3/5:  82%|██████████████████    | 16447/19997 [00:01<00:00, 16457.32it/s]/var/folders/km/967q2mlj1c79my7z2clmws2c0000gn/T/ipykernel_38424/934040620.py:12: RuntimeWarning: overflow encountered in matmul
  x_w = np.matmul(self.weights, x.T)
/var/folders/km/967q2mlj1c79my7z2clmws2c0000gn/T/ipykernel_38424/3338405453.py:10: RuntimeWarning: invalid value encountered in subtract
  e_x = np.exp(x - max)
batch progression 3/5: 100%|██████████████████████| 19997/19997 [00:01<00:00, 16369.76it/s]


[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan]]


batch progression 4/5: 100%|██████████████████████| 19997/19997 [00:01<00:00, 16367.29it/s]

[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan]]


In [146]:
n_vocab = len(vocab)
dat_len = len(stripped_metamorphosis) - 3 #n_prev_words
n_batch = 20000
nn_model = Neural_Network(n_vocab, 32)

for batch in range(dat_len // n_batch):
    x = np.zeroes((n_batch - 3, 3))
    
    progress = tqdm(total=)

SyntaxError: invalid syntax (3395360123.py, line 9)

In [136]:
data = """I've seen what I become, and I cannot let that happen. And for this, you join him? Your destiny can
        change just as quickly as the love in one's heart can fade. Nothing is set in stone. But I will cause
        so much pain. If there is to be balance, what you have seen must be forgotten. If you're not with me,
        then you're my enemy. Only a Sith deals in absolutes. I will do what I must. You will try. It's over,
        Anakin! I have the high ground. You underestimate my power. Don't try it!"""

x = np.array([[[0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1]],
                      [[0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1]],
                      [[1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]],
                      [[0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1]]])

y = [0, 1, 2, 3, 4, 5]

model = Neural_Network(6, 2)
result = model.fit(x, y)
print(result)


Feedforward x-shape:  (4, 3, 6)
(2, 6) (3, 6)
(2, 6) (3, 6)
(2, 6) (3, 6)
(2, 6) (6, 3)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 6)